In [2]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [175]:
training_data = [
    ['H','H','H','H','H'],
    ['H','H','H','L','H'],
    ['H','L','L','M','M'],
    ['H','L','L','H','M'],
    ['L','L','L','L','L'],
]

In [153]:
training_data1 = [
    ['Mumbai','Green', 3, 'Apple','1'],
    ['Mumabi','Yellow', 3, 'Apple','0'],
    ['Kerela','Red', 1, 'Grape','0'],
    ['Kerela','Red', 1, 'Grape','1'],
    ['Mumabi','Yellow', 3, 'Lemon','1'],
]

In [176]:
header1 = ["TEMPHIGH", "TEMPLOW","HUMIDITY","POPULATIO", "label"]

In [41]:
header = ["Color", "Diameter","label"]

In [4]:
def unique_vals(Data, col):
    return set([row[col] for row in Data])

In [6]:
unique_vals(training_data, 1)

{1, 3}

In [9]:
def class_counts(Data):
    counts = {} 
    for row in Data:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [10]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [33]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [36]:
is_numeric(54)

True

In [42]:
class Question:
    def __init__(self, column, value):#1,3
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value 
        else:
            return val == self.value  

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [43]:
Question(0,'red')

Is Color == red?

In [44]:
q=Question(1,3)

In [46]:
q.match(training_data[2])

False

In [25]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [47]:
true_rows, false_rows =partition(training_data, Question(1,3))

In [49]:
false_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [51]:

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [54]:
x=[['apple'],['apple'],['abc'],['abc']]
gini(training_data)

0.6399999999999999

In [57]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    q = float(len(right)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - q * gini(right)

In [58]:
true_rows, false_rows =partition(training_data, Question(1,1))

In [59]:
info_gain(true_rows, false_rows, 0.639)

-0.0009999999999998899

In [60]:
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns
    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [61]:
best_gain, best_question = find_best_split(training_data)
print(best_question)
print(best_gain)

Is Diameter >= 3?
0.37333333333333324


In [62]:
class Leaf:

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [63]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [64]:
def build_tree(rows):

    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [66]:
my_tree = build_tree(training_data)
print(my_tree)

In [67]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [79]:
print_tree(my_tree)

Is Diameter >= 3?
--> True:
  Is Color == Green?
  --> True:
    Predict {'Apple': 1}
  --> False:
    Predict {'Apple': 1, 'Lemon': 1}
--> False:
  Predict {'Grape': 2}


In [68]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [69]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [77]:
testing_data = [
     ['red',1,'apple'],
     ['Yellow',3,'apple']
]

In [78]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: apple. Predicted: {'Grape': '100%'}
Actual: apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
